<a href="https://colab.research.google.com/github/radosty/radosty.github.io/blob/main/extractinggdpppandinternetpenetration2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd

# List of European countries' ISO Alpha-3 codes
european_countries = [
    "ALB", "AND", "ARM", "AUT", "AZE", "BEL", "BIH", "BLR", "BGR", "CHE", "CYP", "CZE",
    "DEU", "DNK", "ESP", "EST", "FIN", "FRA", "GBR", "GEO", "GRC", "HRV", "HUN", "IRL",
    "ISL", "ITA", "KAZ", "KGZ", "LIE", "LTU", "LUX", "LVA", "MCO", "MDA", "MKD", "MLT",
    "MNE", "NLD", "NOR", "POL", "PRT", "ROU", "RUS", "SMR", "SRB", "SVK", "SVN", "SWE",
    "TJK", "TKM", "UKR", "UZB"
]

# Year of interest
year = 2021

# Function to fetch data for a specific indicator
def fetch_data(indicator):
    url = f"https://api.worldbank.org/v2/country/{';'.join(european_countries)}/indicator/{indicator}?date={year}&format=json&per_page=1000"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data and len(data) > 1 and data[1]:
            records = [
                {
                    "Country": record["country"]["value"],
                    "Country Code": record["country"]["id"],
                    "Year": record["date"],
                    "Indicator": indicator,
                    "Value": record["value"],
                }
                for record in data[1]
                if record and "value" in record and record["value"] is not None
            ]
            return pd.DataFrame(records)
        else:
            print(f"No data available for indicator {indicator} in {year}.")
            return pd.DataFrame()
    else:
        print(f"Failed to fetch data for indicator {indicator}. HTTP Status Code: {response.status_code}")
        return pd.DataFrame()

# List of financial inclusion indicators from the Global Findex Database
indicators = [
    "IT.NET.USER.ZS",
    "NY.GDP.PCAP.PP.CD"
]

# Fetch data for all indicators and combine into a single DataFrame
all_data = pd.DataFrame()
for indicator in indicators:
    df = fetch_data(indicator)
    if not df.empty:
        all_data = pd.concat([all_data, df], ignore_index=True)

# Pivot the DataFrame to have indicators as columns
if not all_data.empty:
    final_df = all_data.pivot_table(index=["Country", "Country Code", "Year"], columns="Indicator", values="Value").reset_index()
    # Save to CSV
    filename = "europe_financial_inclusion_2021.csv"
    final_df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")
    print(final_df.head())
else:
    print("No financial inclusion data available for the specified indicators in 2021.")


Data saved to europe_financial_inclusion_2021.csv
Indicator     Country Country Code  Year  IT.NET.USER.ZS  NY.GDP.PCAP.PP.CD
0             Albania           AL  2021         79.3237       16261.804554
1             Andorra           AD  2021         93.8975       58829.850072
2             Armenia           AM  2021         78.6123       16899.070918
3             Austria           AT  2021         92.5292       63421.203587
4          Azerbaijan           AZ  2021         86.0000       20111.375695
